In [183]:
# Necessary imports
import pandas as pd
import numpy as np
import math

In [184]:
# Load data from excel file
stock_transfers_aggregated = pd.read_excel('../data/Get_Stock_Transfers.xlsx')
sales_aggregated = pd.read_excel('../data/Quantity_Sold.xlsx')

In [185]:
# Replace whitespace of column titles with underscores
stock_transfers_aggregated.columns = [title.replace(' ', '_') for title in stock_transfers_aggregated.columns]
sales_aggregated.columns = [title.replace(' ', '_') for title in sales_aggregated.columns]

In [186]:
stock_transfers_aggregated

,Simulation_Round,Simulation_Step,Storage_Location,Quantity
0,12,20,02N,3689
1,12,19,02N,6901
2,12,19,02S,8824
3,12,19,02W,16470
4,12,18,02N,9599
5,12,18,02S,2560
6,12,17,02N,6400
7,12,17,02S,14915
8,12,17,02W,10006
9,12,16,02W,3866


In [187]:
sales_aggregated

,Material_Number,Material_Description,Quantity,Cost,Net_Value
0,CC-F01,500g Nut Muesli,675350,607815.00,3601108.27
1,CC-F02,500g Blueberry Muesli,1192461,1466727.03,5418198.73
2,CC-F04,500g Raisin Muesli,523929,413903.91,2618128.95
3,CC-F11,1kg Nut Muesli,571769,926265.78,2621076.24
4,CC-F12,1kg Blueberry Muesli,482134,1094444.18,2441182.07
5,CC-F13,1kg Strawberry Muesli,344021,784367.88,1809038.77


In [188]:
# Calculate margin/pc of each product
sales_aggregated = sales_aggregated.assign(Margin=lambda x: (x.Net_Value - x.Cost) / x.Quantity)

In [189]:
# Calculate average margin of all products
margin_sum = 0

for index, row in sales_aggregated.iterrows():
    margin_sum += row.Margin
    
average_margin = round(margin_sum / sales_aggregated.shape[0], 2)

In [190]:
average_margin

3.45

In [191]:
# Shipping cost per shipping == 1 000€
# Calculate the shipping quantity to cover the costs of shipping
coverage_quantity = int(math.ceil(1000 / average_margin))

In [192]:
coverage_quantity

290

In [193]:
# Check which stock transfers lost us money
non_coverage_transfers = stock_transfers_aggregated[stock_transfers_aggregated.Quantity <= coverage_quantity].sort_values(by='Quantity')
non_coverage_transfers

,Simulation_Round,Simulation_Step,Storage_Location,Quantity
178,8,20,02N,1
337,5,7,02N,1
401,4,3,02N,1
445,3,7,02N,1


In [206]:
# Transfers to each region
north = stock_transfers_aggregated.loc[stock_transfers_aggregated['Storage_Location'] == '02N']
south = stock_transfers_aggregated.loc[stock_transfers_aggregated['Storage_Location'] == '02S']
west = stock_transfers_aggregated.loc[stock_transfers_aggregated['Storage_Location'] == '02W']
north.info()
south.info()
west.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173 entries, 0 to 515
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Simulation_Round  173 non-null    int64 
 1   Simulation_Step   173 non-null    int64 
 2   Storage_Location  173 non-null    object
 3   Quantity          173 non-null    int64 
 4   Round_Step        173 non-null    object
dtypes: int64(3), object(2)
memory usage: 8.1+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 173 entries, 2 to 516
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Simulation_Round  173 non-null    int64 
 1   Simulation_Step   173 non-null    int64 
 2   Storage_Location  173 non-null    object
 3   Quantity          173 non-null    int64 
 4   Round_Step        173 non-null    object
dtypes: int64(3), object(2)
memory usage: 8.1+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index

In [194]:
groupby = stock_transfers_aggregated
groupby['Round_Step'] = groupby['Simulation_Round'].astype(str) + groupby['Simulation_Step'].astype(str)
groupby = groupby.drop(columns=['Simulation_Round', 'Simulation_Step'])
v = groupby.Round_Step.value_counts()
groupby = groupby.groupby(['Round_Step', 'Storage_Location'], as_index=False).sum()

In [195]:
v

111     6
118     6
113     6
119     5
112     4
115     4
116     4
117     4
114     4
719     3
413     3
414     3
1219    3
412     3
411     3
715     3
410     3
717     3
49      3
48      3
77      3
813     3
44      3
820     3
819     3
818     3
817     3
45      3
815     3
812     3
81      3
810     3
88      3
46      3
86      3
47      3
84      3
82      3
415     3
14      3
417     3
68      3
511     3
512     3
57      3
516     3
55      3
520     3
53      3
62      3
63      3
64      3
65      3
66      3
67      3
420     3
75      3
611     3
612     3
613     3
419     3
615     3
616     3
617     3
619     3
71      3
72      3
73      3
43      3
74      3
92      3
96      3
94      3
38      3
1019    3
36      3
35      3
33      3
1112    3
31      3
1117    3
215     3
1119    3
1120    3
214     3
120     3
110     3
19      3
18      3
127     3
17      3
129     3
1210    3
1211    3
1212    3
1213    3
1214    3
1215    3
16      3
1217    3


In [196]:
groupby

,Round_Step,Storage_Location,Quantity
0,101,02S,13443
1,101,02W,14143
2,1010,02S,7161
3,1010,02W,6271
4,1011,02N,3629
5,1011,02S,2978
6,1011,02W,3053
7,1012,02N,2770
8,1012,02S,3144
9,1012,02W,6587


In [197]:
groupby_location = groupby.groupby(['Round_Step'], as_index=False).count()
groupby_location = groupby_location.drop(columns=['Quantity'], axis=1)

In [198]:
groupby_location

,Round_Step,Storage_Location
0,101,2
1,1010,2
2,1011,3
3,1012,3
4,1013,3
5,1014,2
6,1015,3
7,1016,2
8,1017,2
9,1018,2


In [199]:
groupby_quantity = groupby.groupby(['Round_Step'], as_index=False).sum()

In [208]:
groupby_quantity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Round_Step  199 non-null    object
 1   Quantity    199 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.2+ KB


In [201]:
merged = pd.merge(groupby_location, groupby_quantity, on='Round_Step')

In [202]:
# Change wrong values
merged.loc[merged.Round_Step == '111', 'Storage_Location'] = 6
merged.loc[merged.Round_Step == '118', 'Storage_Location'] = 6
merged.loc[merged.Round_Step == '113', 'Storage_Location'] = 6
merged.loc[merged.Round_Step == '119', 'Storage_Location'] = 5
merged.loc[merged.Round_Step == '112', 'Storage_Location'] = 4
merged.loc[merged.Round_Step == '115', 'Storage_Location'] = 4
merged.loc[merged.Round_Step == '116', 'Storage_Location'] = 4
merged.loc[merged.Round_Step == '117', 'Storage_Location'] = 4
merged.loc[merged.Round_Step == '114', 'Storage_Location'] = 4

In [203]:
merged

,Round_Step,Storage_Location,Quantity
0,101,2,27586
1,1010,2,13432
2,1011,3,9660
3,1012,3,12501
4,1013,3,22311
5,1014,2,16232
6,1015,3,34562
7,1016,2,6533
8,1017,2,27112
9,1018,2,17897


In [204]:
non_coverage_transfers_fixcosts = merged[merged.Quantity <= (merged['Storage_Location'] * 293) + 7091].sort_values(by='Quantity')
non_coverage_transfers_fixcosts

,Round_Step,Storage_Location,Quantity
192,93,1,1965
177,85,1,2475
113,513,1,2495
179,87,1,2646
120,54,1,2781
135,618,1,2828
189,917,1,3067
12,1020,1,3117
147,711,1,3250
53,124,1,3264


In [205]:
non_coverage_transfers_fixcosts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 192 to 54
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Round_Step        30 non-null     object
 1   Storage_Location  30 non-null     int64 
 2   Quantity          30 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 960.0+ bytes
